## #Tag Matters
#### To understand how tags play the role by using it on NYTimes daily and how people really react about the topic by looking at other media platforms through NYTimes tags.

In the last twelve months, 563 of tags has been appeared every single months. Espetially, the specific terms included tags are shown that there were always happened 

- General tags: 'Global Warming', 'Prices (Fares, Fees and Rates)', 'Computer Security', 'Depression (Mental)'
- Countries' name: 'China', 'Iran', 'North Korea', 'Russia'
- Specific terms included: 'Hong Kong Protests (2019)', 'Civil Rights Movement (1954-68)', 'Tax Cuts and Jobs Act (2017)', '#MeToo Movement'

In [12]:
import sys
sys.executable

'/usr/local/opt/python/bin/python3.7'

In [13]:
sys.path

['/Users/hh/Documents/Pratt/Adv.ProjectsinVis/Monthly-Frequency-of-NYTimes-Tag',
 '/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python37.zip',
 '/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python3.7',
 '/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload',
 '',
 '/Users/hh/Library/Python/3.7/lib/python/site-packages',
 '/usr/local/lib/python3.7/site-packages',
 '/usr/local/lib/python3.7/site-packages/IPython/extensions',
 '/Users/hh/.ipython']

In [1]:
import json
import requests
import pandas as pd
import numpy as np
import operator
import time
import praw
import nltk
import configparser
import re

from apiclient import discovery
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta

from pytrends.request import TrendReq
from praw.models import MoreComments
from googleapiclient import discovery
from textblob import TextBlob
from pandas.io.json import json_normalize

import pprint
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

from sqlalchemy.orm import sessionmaker

In [3]:
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.sql import *
from sqlalchemy.orm import relationship

In [4]:
# Use CofigParser to safely store the password or key
config = configparser.ConfigParser()
config.read('key_pair.ini')

Times_key = config['Times']['key']

Reddit_client_id = config['Reddit']['client_id']
Reddit_client_secret = config['Reddit']['client_secret']
Reddit_username = config['Reddit']['username']
Reddit_password = config['Reddit']['password']
Reddit_user_agent = config['Reddit']['user_agent']

Youtube_service_name = config['Youtube']['YOUTUBE_API_SERVICE_NAME']
Youtube_API_version = config['Youtube']['YOUTUBE_API_VERSION']
Youtube_developer_key = config['Youtube']['DEVELOPER_KEY']

### Storing Data using SQLalchemy

In [5]:
engine = create_engine('sqlite:///monthly_times_tags.db', echo=True)
Base = declarative_base()

In [6]:
class TimesTable(Base): 
    __tablename__ = 'times'
    __table_args__ = {'extend_existing': True}

    id = Column(Integer, primary_key=True)
    tag = Column(String(255))
    periodeM = Column(String(25))
    frequency = Column(Integer)
    title = Column(String(255))
    date = Column(String(25))
    url = Column(String(255))
    img_URL = Column(String(255))

class GoogleTable(Base): 
    __tablename__ = 'google'
    __table_args__ = {'extend_existing': True}

    id = Column(Integer, primary_key=True)
    trendDate = Column(PickleType)
    busiest = Column(String(25))
    trendIndex = Column(PickleType)
    fk_times = Column(Integer, ForeignKey("times.id", ondelete='CASCADE'), nullable=False)
    googleNTag = relationship('TimesTable')


class RedditTable(Base): 
    __tablename__ = 'reddit'
    __table_args__ = {'extend_existing': True}

    id = Column(Integer, primary_key=True)
    tag = Column(String(255))
    commentCount = Column(Integer)
    
class YoutubeTable(Base): 
    __tablename__ = 'youtube'
    __table_args__ = {'extend_existing': True}

    id = Column(Integer, primary_key=True)
    tag = Column(String(255))
    viewCount = Column(Integer)
    commentCount = Column(Integer)
    likeCount = Column(Integer)

# TimesTable.__table__.create(bind=engine, checkfirst=True)
# GoogleTable.__table__.create(bind=engine, checkfirst=True)
# RedditTable.__table__.create(bind=engine, checkfirst=True)
# YoutubeTable.__table__.create(bind=engine, checkfirst=True)

### New York Times API
https://developer.nytimes.com/apis

Archive API
- Finds all articles's metadata from archive APi (about 7000 every month): title, section_name, period, url, word_count, keywords

In [7]:
def cleaning_tag(tag):
    if tag in ['School Shootings and Armed Attacks']:
        tag = 'School Shootings'
    if tag in ['Midland-Odessa, Tex, Shooting (2019)', 'Dayton, Ohio, Shooting (2019)', 'El Paso, Tex, Shooting (2019)', 'El Paso', 'Dayton', 'Shooting (2019)', 'Tex']:
        tag = 'Shootings'
    if tag in ['Homosexuality and Bisexuality']:
        tag = 'Homosexuality'
    if tag in ['Biden, Joseph R Jr']:
        tag = 'Joe Biden'
    if tag in ['Floyd, George (d 2020)']:
        tag = 'George Floyd Protests (2020)'
    if tag in ['Trump-Ukraine Whistle-blower Complaint and Impeachment Inquiry']:
        tag = 'Trump Ukraine Whistle blower'
    if 'Security Act (2020)' in tag:
        tag = 'Coronavirus Aid Relief and Economic Security Act'
    if 'Suleimani' in tag:
        tag = 'Qasem Soleimani'
    if 'Homicides' in tag:
        tag ='Homicides'
    if 'Buttigieg' in tag:
        tag = 'Pete Buttigieg'
    if 'Police Brutality' in tag:
        tag = 'Police Brutality'
    if 'Shutdowns' in tag:
        tag = 'Shutdowns'
    if 'Quarantine' in tag:
        tag = 'Quarantines'
    if 'Deaths' in tag:
        tag = 'Deaths'
    if 'Russian Interference in 2016' in tag:
        tag = 'Russian interference in the 2016 United States elections'
    if 'Syria' in tag:
        tag = 'Syria'    
    if tag in ['Presidential Election of 2020', 'Elections', 'United States Politics and Government', 'New York City', 'United States', 'Politics and Government', 'Trump, Donald J']:
        tag = ''
    return tag

In [8]:
def get_NYTimes_metadata():

    # To get most frequently used tag in NYTimes
    today = str(datetime.today())
    monthly_archive = {}
    frequent_tags_archive = {}

    for yy in reversed(range(2019, int(today[:4]) + 1)):
        if str(yy) == today[:4]:
            ends = int(today[5:7]) + 1
            start = 1
        else:
            ends = 13
            if '0' in today[5:7]:
                start = int(today[6:7]) + 1
            else:
                start = int(today[5:7]) + 1

        for mm in reversed(range(start,ends)):
            if len(str(mm)) == 1:
                date_not_duplicate = str(yy) + '-0' + str(mm)
            else:
                date_not_duplicate = str(yy) + '-' + str(mm)
            if date_not_duplicate not in monthly_archive:
                parameters = {'api-key': Times_key}           
                archived_Url = 'https://api.nytimes.com/svc/archive/v1/'+ str(yy) +'/'+ str(mm) +'.json'
                archives = requests.get(archived_Url, params=parameters).json()

                monthly_article = []
                for a in archives['response']['docs']:
                    articles = {}
                    articles['title'] = a['headline']['main']
                    articles['pub_date'] = a['pub_date'][:10]
                    articles['url'] = a['web_url']
                    
                    if len(a['multimedia']) !=0 and a['multimedia'][0]['url']:
                        articles['thm_img'] = 'https://static01.nyt.com/' + a['multimedia'][0]['url']
                    else:
                        articles['thm_img'] = 'no_image_found'
                    
#                     cleaning_article = map(cleaning_tag,  [''.join(tag['value']) for tag in a['keywords']])
#                     articles['tags'] = [changed_tag if ', ' not in changed_tag else changed_tag.replace(", ", " ") for changed_tag in cleaning_article]
                    merging_article = [''.join(tag['value']) for tag in a['keywords']]
                    articles['tags'] = [changed_tag if ', ' not in changed_tag else changed_tag.replace(", ", " ") for changed_tag in merging_article]
                    
                    monthly_article.append(articles)
                    
                if len(str(mm)) == 1:
                    mm = '0' + str(mm)
                monthly_archive[str(yy)+'-'+str(mm)] = monthly_article
                
                count_tag = {}
                for m in monthly_article:
                    for tag in m['tags']:
                        if tag is not '':
                            if tag in count_tag:
                                count_tag[tag] += 1
                            else:
                                count_tag[tag] = 1   

                # This variable is what we want to get in NYTimes
                tags_with_frequency = sorted(count_tag.items(),key=operator.itemgetter(1),reverse=True)[:20]
                frequent_tags_archive[str(yy)+'-'+str(mm)] = tags_with_frequency

    return monthly_archive, frequent_tags_archive

In [9]:
monthly_archive = get_NYTimes_metadata()[0]
frequent_tags_archive = get_NYTimes_metadata()[1]

In [10]:
frequent_tags_archive

{'2020-07': [('Coronavirus (2019-nCoV)', 1214),
  ('Trump Donald J', 571),
  ('United States Politics and Government', 548),
  ('Coronavirus Reopenings', 457),
  ('Presidential Election of 2020', 395),
  ('Black People', 256),
  ('Blacks', 256),
  ('Books and Literature', 243),
  ('United States', 219),
  ('Politics and Government', 190),
  ('George Floyd Protests (2020)', 189),
  ('Quarantines', 189),
  ('Quarantine (Life and Culture)', 187),
  ('Race and Ethnicity', 186),
  ('Black Lives Matter Movement', 182),
  ('Discrimination', 177),
  ('Movies', 151),
  ('Primaries and Caucuses', 150),
  ('New York City', 148),
  ('Television', 142)],
 '2020-06': [('Coronavirus (2019-nCoV)', 1735),
  ('George Floyd Protests (2020)', 1170),
  ('Presidential Election of 2020', 889),
  ('United States Politics and Government', 803),
  ('Black People', 777),
  ('Trump Donald J', 760),
  ('Black Lives Matter Movement', 639),
  ('Primaries and Caucuses', 636),
  ('Coronavirus Reopenings', 633),
  ('El

In [36]:
count_tag = {}
for period in frequent_tags_archive.values():
    for tag in period:
        if tag[0] is not '':
            if tag[0] in count_tag:
                count_tag[tag[0]] += 1
            else:
                count_tag[tag[0]] = 1 
tags_with_frequency = sorted(count_tag.items(),key=operator.itemgetter(1),reverse=True)
tags_with_frequency

[('Presidential Election of 2020', 12),
 ('United States Politics and Government', 12),
 ('Trump Donald J', 12),
 ('Books and Literature', 12),
 ('Politics and Government', 12),
 ('New York City', 12),
 ('Movies', 10),
 ('Television', 10),
 ('United States', 9),
 ('Democratic Party', 9),
 ('Real Estate and Housing (Residential)', 8),
 ('United States International Relations', 7),
 ('Deaths (Obituaries)', 6),
 ('Coronavirus (2019-nCoV)', 5),
 ('Primaries and Caucuses', 5),
 ('Trump-Ukraine Whistle-blower Complaint and Impeachment Inquiry', 5),
 ('Impeachment', 5),
 ('Quarantines', 4),
 ('United States Economy', 4),
 ('Epidemics', 4),
 ('Biden Joseph R Jr', 4),
 ('China', 4),
 ('Elections', 3),
 ('Discrimination', 3),
 ('Shutdowns (Institutional)', 3),
 ('Sanders Bernard', 3),
 ('Republican Party', 3),
 ('Ukraine', 3),
 ('Art', 3),
 ('Black People', 2),
 ('Coronavirus Reopenings', 2),
 ('Demonstrations Protests and Riots', 2),
 ('Race and Ethnicity', 2),
 ('Quarantine (Life and Culture)'

In [614]:
def store_times(monthly_archive, frequent_tags_archive):
    multi_articles = {}
    been_used = []
    for periode in frequent_tags_archive:
        print('='*20)
        print('periode : ', periode)     
        print('='*20)
        for top_tag in frequent_tags_archive[periode]:
            for i, each in enumerate(reversed(monthly_archive[periode])):
                if top_tag[0] in multi_articles and periode in multi_articles[top_tag[0]]:
                    pass                             
                else:
                    if len(each['tags']) != 0 and top_tag[0] in each['tags'] and each['title'] not in been_used:
                        try:
                            multi_articles[top_tag[0]] = periode
                            been_used.append(each['title'])
                            
                        except Exception as e:
                            print(" = Unable insert_TimesTag: ", top_tag[0], periode, " =", e)
                            pass   
    return #multi_articles

In [615]:
store_times(monthly_archive, frequent_tags_archive)

periode :  2020-06
Coronavirus (2019-nCoV)
How to Clean Up After Others
2020-06-16
George Floyd Protests (2020)
The ‘Poisoned’ Shake Shack Milkshakes and an N.Y.P.D. on Edge
2020-06-16
Presidential Election of 2020
Biden and D.N.C. Raised Nearly $81 Million in May, a New High
2020-06-15
Black People
The Unexpected Beauty of ‘Dog Vomit’ and Other Slime
2020-06-16
Police Brutality
Man Charged With Sending Racist Threats to Philadelphia Police Leader
2020-06-17
Black Lives Matter Movement
Who’s a Bot? Who’s Not?
2020-06-16
Quarantines
Riding Out Quarantine With a Chatbot Friend: ‘I Feel Very Connected’
2020-06-16
Coronavirus Reopenings
I’m a Critical Care Doctor. I’m Tired, I’m Mourning, I’m Bracing for More.
2020-06-16
Demonstrations Protests and Riots
How a Once-Loathed Brooklyn Arena Became a Protest Epicenter
2020-06-16
Race and Ethnicity
Why Andrea Jenkins Is Somber, but Optimistic
2020-06-16
periode :  2020-05
Coronavirus (2019-nCoV)
Grateful for Aid, but Worried About What Comes Ne

In [509]:
dayDates = []
today = datetime.now()
dayDates.append(today.strftime("%m%d%Y"))
for i in range(0,14):
    day = today + timedelta(days=i)
    print(day)

2020-06-11 14:50:03.300148
2020-06-12 14:50:03.300148
2020-06-13 14:50:03.300148
2020-06-14 14:50:03.300148
2020-06-15 14:50:03.300148
2020-06-16 14:50:03.300148
2020-06-17 14:50:03.300148
2020-06-18 14:50:03.300148
2020-06-19 14:50:03.300148
2020-06-20 14:50:03.300148
2020-06-21 14:50:03.300148
2020-06-22 14:50:03.300148
2020-06-23 14:50:03.300148
2020-06-24 14:50:03.300148


In [512]:
datetime.today()

datetime.datetime(2020, 6, 11, 14, 50, 35, 789738)

In [525]:
today = datetime.today()
pre_month = str(today.replace(day=1) - timedelta(days=22))
pre_month

'2020-05-10 14:52:52.284450'

### Top Tags with Frequency per each Month

In [648]:
def call_monthly_top_tags(frequent_tags_archive, number):
    monthly_top_tags_with_frequency = {}
    for time_period in frequent_tags_archive.keys():
        tag_only = []
        for each in frequent_tags_archive[time_period][:number]: 
            tag_only.append(each[0])
            monthly_top_tags_with_frequency[time_period] = tag_only
    return monthly_top_tags_with_frequency

In [649]:
monthly_top10_tags = call_monthly_top_tags(frequent_tags_archive, 5)
monthly_top10_tags

{'2020-06': ['Coronavirus (2019-nCoV)',
  'George Floyd Protests (2020)',
  'Presidential Election of 2020',
  'Black People',
  'Police Brutality'],
 '2020-05': ['Coronavirus (2019-nCoV)',
  'Quarantines',
  'Shutdowns',
  'Coronavirus Reopenings',
  'Presidential Election of 2020'],
 '2020-04': ['Coronavirus (2019-nCoV)',
  'Quarantines',
  'Parenting',
  'Presidential Election of 2020',
  'Epidemics'],
 '2020-03': ['Coronavirus (2019-nCoV)',
  'Quarantines',
  'Presidential Election of 2020',
  'Epidemics',
  'Joe Biden'],
 '2020-02': ['Presidential Election of 2020',
  'Coronavirus (2019-nCoV)',
  'Sanders Bernard',
  'China',
  'Epidemics'],
 '2020-01': ['Presidential Election of 2020',
  'Trump Ukraine Whistle blower',
  'Iran',
  'Qasem Soleimani',
  'China'],
 '2019-12': ['Presidential Election of 2020',
  'Trump Ukraine Whistle blower',
  'Demonstrations Protests and Riots',
  'China',
  'Homicides'],
 '2019-11': ['Trump Ukraine Whistle blower',
  'Presidential Election of 202

### Unique Tags list 

In [654]:
def call_unique_tag_list(monthly_top10_tags):
    count_tag = {}
    for periode in monthly_top10_tags:
        for tag in monthly_top10_tags[periode]:
            if tag is not '' and tag in count_tag:
                count_tag[tag] += 1
            else:
                count_tag[tag] = 1  
    unique_tag_list = sorted(count_tag.items(),key=operator.itemgetter(1),reverse=True)
    unique_tag_list = [tags_f for tags_f in unique_tag_list] #if tags_f[1] > 9]
    return unique_tag_list

In [655]:
unique_tag_list = call_unique_tag_list(monthly_top10_tags)
unique_tag_list

[('Presidential Election of 2020', 12),
 ('Coronavirus (2019-nCoV)', 5),
 ('China', 5),
 ('Trump Ukraine Whistle blower', 5),
 ('Quarantines', 3),
 ('Epidemics', 3),
 ('Joe Biden', 3),
 ('Ukraine', 3),
 ('Demonstrations Protests and Riots', 2),
 ('Blacks', 2),
 ('Immigration and Emigration', 2),
 ('George Floyd Protests (2020)', 1),
 ('Black People', 1),
 ('Police Brutality', 1),
 ('Shutdowns', 1),
 ('Coronavirus Reopenings', 1),
 ('Parenting', 1),
 ('Sanders Bernard', 1),
 ('Iran', 1),
 ('Qasem Soleimani', 1),
 ('Homicides', 1),
 ('Warren Elizabeth', 1),
 ('Syria', 1),
 ('Fashion and Apparel', 1),
 ('Shootings', 1),
 ('Race and Ethnicity', 1)]

### Popular Tags that appear on every single month

In [627]:
def call_tags_appeared_every_month(monthly_archive):
    dicts = {}
    for periode in monthly_archive:
        tag_arr = []
        dicts[periode] = tag_arr
        for each in reversed(monthly_archive[periode]):
            for t in each['tags']:
                for string in re.split(r',(?=")', t):
                    if string is not '' and string not in tag_arr:
                        tag_arr.append(', '.join(string.split(', ')))   
    count_tag = {}
    for tag_arr in dicts.values():
        for tag in tag_arr:
            if tag is not '':
                if tag in count_tag:
                    count_tag[tag] += 1
                else:
                    count_tag[tag] = 1   
    monthly_unique_tags = sorted(count_tag.items(),key=operator.itemgetter(1),reverse=True)  
    monthly_unique_tags = [tags_f for tags_f in monthly_unique_tags if tags_f[1] > 11]
    return monthly_unique_tags  

In [628]:
tags_appeared_every_month = call_tags_appeared_every_month(monthly_archive)
tags_appeared_every_month

[('Protective Clothing and Gear', 12),
 ('Waste Materials and Disposal', 12),
 ('Fashion and Apparel', 12),
 ('Smoking and Tobacco', 12),
 ('Forests and Forestry', 12),
 ('Classified Information and State Secrets', 12),
 ('Police', 12),
 ('Manhattan (NYC)', 12),
 ('Defense and Military Forces', 12),
 ('International Relations', 12),
 ('China', 12),
 ('India', 12),
 ('Decisions and Verdicts', 12),
 ('Homosexuality', 12),
 ('Transgender and Transsexuals', 12),
 ('Conservatism (US Politics)', 12),
 ('Workplace Hazards and Violations', 12),
 ('Supreme Court (US)', 12),
 ('Wines', 12),
 ('Classical Music', 12),
 ('Recipes', 12),
 ('Customs (Tariff)', 12),
 ('Affordable Housing', 12),
 ('Pop and Rock Music', 12),
 ('Police Brutality', 12),
 ('Cyberharassment', 12),
 ('Race and Ethnicity', 12),
 ('Federal Bureau of Investigation', 12),
 ('Philadelphia (Pa)', 12),
 ('Massachusetts', 12),
 ('Ex-Convicts', 12),
 ('Voter Registration and Requirements', 12),
 ('Voting and Voters', 12),
 ('Executiv

### Pytrends API
https://pypi.org/project/pytrends/

Sample codes to see Google doesn't block my IP

In [643]:
pytrends = TrendReq(hl='en-US', tz=360, timeout=(10,25))
tag_arr = ['Coronavirus (2019-nCoV)']
pytrends.build_payload(tag_arr, cat=0, timeframe='2020-04-01 ' + str(datetime.now())[:10], geo='', gprop='')
time.sleep(2)
df = pytrends.interest_over_time().reset_index()
df.head()

,date,Coronavirus (2019-nCoV),isPartial
0,2020-04-01,52,False
1,2020-04-02,29,False
2,2020-04-03,73,False
3,2020-04-04,45,False
4,2020-04-05,45,False


In [260]:
# Get a unique tag collection for the search query
def get_trends_Tags(monthly_top10_tags): 
    today = str(datetime.today())
    pytrends = TrendReq(hl='en-US', tz=360)
    monthly_interests = {}
    
    for period in monthly_top10_tags:
        data = []
        for tag in monthly_top10_tags[period]:
            tag_arr = []
            tag_arr.append(tag)
            
            interest_over_time = {}
            try:
                pytrends.build_payload(tag_arr, cat=0, timeframe = '2019-05-01 '+ today[:10], geo='', gprop='')
                df = pytrends.interest_over_time().reset_index()
            except Exception as e:
                print(tag_arr, ', and which reason? ', e)
                pass
            
            interest_over_time['Tag'] = tag
            try:
                interest_over_time['StartDate'] = one_month_ago
                interest_over_time['EndDate'] = one_month_later
                interest_over_time['Rate'] = [rate for rate in df[tag]]
            except Exception as e:
                pass
            data.append(interest_over_time)     
        monthly_interests[period] = data
    return monthly_interests

In [261]:
monthly_interests = get_trends_Tags(monthly_top10_tags_with_frequency)

{'2020-06': ['Coronavirus (2019-nCoV)', 'George Floyd Protests (2020)', 'Police Brutality', 'Black People', 'Demonstrations Protests and Riots', 'Quarantines', 'Black Lives Matter Movement', 'Coronavirus Reopenings', 'Race and Ethnicity', 'California'], '2020-05': ['Coronavirus (2019-nCoV)', 'Quarantines', 'Shutdowns', 'Coronavirus Reopenings', 'Black People', 'Epidemics', 'Children and Childhood', 'Biden Joseph R Jr', 'Parenting', 'Labor and Jobs'], '2020-04': ['Coronavirus (2019-nCoV)', 'Quarantines', 'Parenting', 'Epidemics', 'Children and Childhood', 'Coronavirus Aid Relief and Economic Security Act (2020)', 'Shutdowns', 'Hospitals', 'Babies and Infants', 'Labor and Jobs'], '2020-03': ['Coronavirus (2019-nCoV)', 'Quarantines', 'Epidemics', 'Biden Joseph R Jr', 'Shutdowns', 'Sanders Bernard', 'California', 'Hospitals', 'Tests (Medical)', 'Centers for Disease Control and Prevention'], '2020-02': ['Coronavirus (2019-nCoV)', 'Sanders Bernard', 'Epidemics', 'Biden Joseph R Jr', 'Bloombe

KeyboardInterrupt: 

In [140]:
datetime.date.today()

datetime.date(2020, 6, 10)

In [156]:
str(datetime.date.today().replace(day=1) - datetime.timedelta(days=31))

'2020-05-01'

### Youtube API
https://developers.google.com/youtube/v3/docs/search/list?hl=en_US

In [9]:
unique_tag_list = [('Joe Biden', 9),
 ('California', 6)]

In [16]:
def call_Youtube(unique_tag_list):
    # creating Youtube Resource Object 
    youtube_object = discovery.build(Youtube_service_name, Youtube_API_version, developerKey = Youtube_developer_key)

    max_results = 2
    youtube_metadata = []
    
    for tag_frequency in unique_tag_list[:2]:
        viewCount = 0
        commentCount = 0
        likeCount = 0
        turned_off_comments = 0
        
        video = {}
        contain_stats = {}
        
        time.sleep(2)
        search_tags = youtube_object.search().list(q = tag_frequency[0], part = "id, snippet", order = 'relevance', maxResults = max_results, publishedAfter = "2020-01-01T00:00:00Z").execute() 
        time.sleep(2)
        
        for item in search_tags.get("items", []):
#             pprint.pprint(item)
            try:
                url = 'www.youtube.com/watch?v=' + item['id']['videoId']
                title = item['snippet']['title']
                img_url = item['snippet']['thumbnails']['medium']
                stats = youtube_object.videos().list(part='statistics, snippet', id=item["id"]["videoId"]).execute()
                time.sleep(2)
                viewCount = int(stats.get("items", [])[0]['statistics']['viewCount'])
#                 print('viewCount, ', viewCount)
                likeCount =  int(stats.get("items", [])[0]['statistics']['likeCount'])
#                 print('likeCount, ', likeCount) 
                commentCount = int(stats.get("items", [])[0]['statistics']['commentCount'])
#                 print('commentCount, ', commentCount)

            except Exception as e:
                print(" = Unable search Youtube : ", tag_frequency[0], " =", e)
                if e == 'commentCount':
                    new_likeCount = 0 #'Comments are turned off'
                    contain_stats['turned_off_comments'] = 0
                elif e == 'likeCount':
                    new_likeCount = 0
                elif e == 'viewCount':
                    new_viewCount = 0

            if 'viewCount' in contain_stats:
                contain_stats['viewCount'] = contain_stats['viewCount'] + viewCount
            else:
                contain_stats['viewCount'] = viewCount
            if 'commentCount' in contain_stats:
                contain_stats['commentCount'] = contain_stats['commentCount'] + commentCount
            else:
                contain_stats['commentCount'] = commentCount
            if 'likeCount' in contain_stats:
                contain_stats['likeCount'] = contain_stats['likeCount'] + likeCount
            else:
                contain_stats['likeCount'] = likeCount
            if 'turned_off_comments' in contain_stats:
                contain_stats['turned_off_comments'] = contain_stats['turned_off_comments'] + 1
            
            if tag_frequency[0] in video.keys():
                pass
            else:
                print('* ', tag_frequency[0])
                print('* ', title)
                print('* ', contain_stats['viewCount'])
                print('-'*20, )
                video['tag'] = tag_frequency[0]
                video['url'] = url
                video['title'] = title
                video['img_url'] = img_url
                video['stats'] = contain_stats
                
        youtube_metadata.append(video)
        
    return youtube_metadata

In [17]:
youtube_metadata = call_Youtube(unique_tag_list)

*  Joe Biden
*  Biden Leads Trump Nationally In Two New Polls | Morning Joe | MSNBC
*  111947
--------------------
*  Joe Biden
*  Jake Tapper on the Importance of Joe Biden’s VP Pick
*  115375
--------------------
*  California
*  Usher - California (from Songland) (Official Music Video) ft. Tyga
*  121464
--------------------
*  California
*  California orders everyone wear masks. So what does that mean?
*  122628
--------------------


In [18]:
youtube_metadata

[{'tag': 'Joe Biden',
  'url': 'www.youtube.com/watch?v=TqsXqU9pRF8',
  'title': 'Jake Tapper on the Importance of Joe Biden’s VP Pick',
  'img_url': {'url': 'https://i.ytimg.com/vi/TqsXqU9pRF8/mqdefault.jpg',
   'width': 320,
   'height': 180},
  'stats': {'viewCount': 115375, 'commentCount': 2312, 'likeCount': 3892}},
 {'tag': 'California',
  'url': 'www.youtube.com/watch?v=ZJqmOgfoGVk',
  'title': 'California orders everyone wear masks. So what does that mean?',
  'img_url': {'url': 'https://i.ytimg.com/vi/ZJqmOgfoGVk/mqdefault.jpg',
   'width': 320,
   'height': 180},
  'stats': {'viewCount': 122628, 'commentCount': 796, 'likeCount': 10795}}]

In [642]:
for each_youtube in youtube_metadata:
    unique_tag = list(each_youtube.keys())[0]
    print(unique_tag)

url
url


## Not using anymore

- Top Stories API: (about 30 articles by 7 days): title, pub_date, url, section, des_facet, geo_facet
- Newswire API: Finds by Archive API's url : des_facet
- Most Popular API
- Top Stories API, Times Tags API, Community API

### Reddit API
https://praw.readthedocs.io/en/latest/

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [246]:
def get_reddit_comments(overall_Unique_tag_with_frequency):
    
    reddit = praw.Reddit(client_id = Reddit_client_id,
                         client_secret = Reddit_client_secret,
                         username = Reddit_username,
                         password = Reddit_password,
                         user_agent = Reddit_user_agent)
#     times_metadata = {'2020-04':['Coronavirus (2019-nCoV)', 'Quarantines', 'Parenting'], '2019-04': ['China', 'Epidemics', 'Deaths (Fatalities)']}

    reddit_metadata = []
    for tag_frequency in overall_Unique_tag_with_frequency:
        subreddit = reddit.subreddit('all')
        time.sleep(2)

        each_tag = {}
        for i, post in enumerate(subreddit.search(tag_frequency[0], sort='relevance', syntax='lucene', limit=3)):
            count = 0;
            if not post.stickied:
                each_tag['Tag'] = tag_frequency[0]
#                 print('* *',tag_frequency[0])
#                 each_tag['Title'] = post.title
#                 each_tag['CreatedAt'] = datetime.fromtimestamp(post.created_utc).isoformat()[:10]
#                 each_tag['DiscussionAbout'] = post.url
#                 print('* * *', post.url)
                each_tag_comments = post.comments.list()
                comments_arr = []
                for comment in each_tag_comments:
                    if isinstance(comment, MoreComments):
                        continue
                    comments_arr.append(comment.body)
                    count = count + len(comments_arr)
                each_tag['comments'] = count

        reddit_metadata.append(each_tag)
                    
    return reddit_metadata

In [173]:
def get_reddit_Posts(times_metadata):   
#     result = {}
#     for time_period in times_metadata:
#         data = []
#         for each in times_metadata[time_period]:     
#             data.append(each['Tag'])
#             result[time_period] = data  
            
    reddit = praw.Reddit(client_id = Reddit_client_id,
                         client_secret = Reddit_client_secret,
                         username = Reddit_username,
                         password = Reddit_password,
                         user_agent = Reddit_user_agent)
    result = {'2020-04':['Coronavirus (2019-nCoV)', 'Quarantines', 'Parenting'], '2019-04': ['China', 'Epidemics', 'Deaths (Fatalities)']}
 
    reddit_metadata = []
    for period in result:
        print(period + ': ', len(result[period]))
        reddit_posts = {}
        add_post_arr = []
        for tag in result[period]:
            subreddit = reddit.subreddit('all')
            time.sleep(2)
            for i, post in enumerate(subreddit.search(tag, sort='relevance', syntax='lucene', limit=100)):
                if not post.stickied:
                    each_tag = {}
                    each_tag['Tag'] = tag
#                     print('--1--',tag)
                    each_tag['Title'] = post.title
                    each_tag['CreatedAt'] = datetime.fromtimestamp(post.created_utc).isoformat()[:10]
                    each_tag['DiscussionAbout'] = post.url
                    each_tag['Ups'] = post.ups
                    each_tag['Downs'] = post.downs
                    add_post_arr.append(each_tag)
        reddit_posts[period] = add_post_arr
        reddit_metadata.append(reddit_posts)
#         print('--2--',reddit_metadata)
    
#     add_tag = {}
#     for item in reddit_metadata:
#         for period in item: 
#             monthly_posts = []
#             byTime = {}
            
#             for each_post in item[period]:
#                 each_post['CreatedAt'] = each_post['CreatedAt'][:7]

#                 if each_post['Tag'] in add_tag:
#                     if each_post['CreatedAt'] in byTime:
#                         byTime[each_post['CreatedAt']][0] += 1
#                         byTime[each_post['CreatedAt']][1] += each_post['Ups']
#                     else:
#                         add_freq_votes = []
#                         byTime[each_post['CreatedAt']] = add_freq_votes
#                         add_freq_votes.append(1)
#                         add_freq_votes.append(each_post['Ups'])
#                 else:
#                     add_tag[each_post['Tag']] = byTime
#                     if each_post['CreatedAt'] in byTime:
#                         byTime[each_post['CreatedAt']][0] += 1
#                         byTime[each_post['CreatedAt']][1] += each_post['Ups']
#                     else:
#                         add_freq_votes = []
#                         byTime[each_post['CreatedAt']] = add_freq_votes
#                         add_freq_votes.append(1)
#                         add_freq_votes.append(each_post['Ups'])
# #                     print('--1--', add_tag)
#             monthly_posts.append(add_tag)
# #             print('--2--', monthly_posts)
    
# #     print('--3--', monthly_posts)
    return reddit_metadata

In [138]:
frequent_tags_archive = {'2020-04': [('Coronavirus (2019-nCoV)', 3591),
  ('Quarantines', 822)]}

In [ ]:
 # reddit = praw.Reddit(client_id = Reddit_client_id,
    #                      client_secret = Reddit_client_secret,
    #                      username = Reddit_username,
    #                      password = Reddit_password,
    #                      user_agent = Reddit_user_agent)
    # reddit_metadata = []
    
    # # Store numbers of each Reddit's comments into DB
    # try:
    #     print('Check tag existing on Reddit: ', tag_frequency[0])
    #     subreddit = reddit.subreddit('all')
    #     time.sleep(2)

    #     each_tag = {}
    #     for post in subreddit.search(tag_frequency[0], sort='relevance', syntax='lucene', time_filter='month', limit=max_results):
    #         count = 0;
    #         if not post.stickied:
    #             each_tag['Tag'] = tag_frequency[0]
    #             each_tag_comments = post.comments.list()
    #             comments_arr = []
    #             for comment in each_tag_comments:
    #                 if isinstance(comment, MoreComments):
    #                     continue
    #                 comments_arr.append(comment.body)
    #                 count = count + len(comments_arr)
    #             each_tag['commentCount'] = count
    # except Exception as e:
    #     print(" = Unable search Reddit : ", tag_frequency[0], " =")
    #     pass
    
    # reddit_metadata.append(each_tag)

    # print('= reddit_metadata =', reddit_metadata)

    # for each_reddit in reddit_metadata:    
    #     try:
    #         insert_Reddit = RedditTable(
    #             tag=each_reddit['tag'],
    #             commentCount=each_reddit['commentCount']
    #         )
    #         session.add(insert_Reddit)

    #     except Exception as e:
    #         print(" = Unable insert_Reddit to DB : ", tag_frequency[0], " =")
    #         pass

In [ ]:
def get_sentimental_chart(reddit_metadata):
    tags_metadata = []
    for single_tag in reddit_metadata:
        tag_sentiment = {}
        avg_polarity = 0
        pos_count = 0
        neg_count = 0
        neutral_count = 0
        for single_comment in single_tag['Comments']: 
            
            # Get Word Sentimental analysis
            blobed_word = TextBlob(single_comment)
            if blobed_word.sentiment.polarity > 0:
                pos_count += 1
            elif blobed_word.sentiment.polarity == 0:
                neutral_count += 1
            else:
                neg_count += 1      
            avg_polarity += blobed_word.sentiment.polarity
            
            tag_sentiment['tag'] = single_tag['Tag']    
            tag_sentiment['avg_polarity'] = avg_polarity / len(single_tag['Comments'])
            tag_sentiment['pos_count'] = pos_count
            tag_sentiment['neg_count'] = neg_count
            tag_sentiment['neutral_count'] = neutral_count
            
            # Get Adjective words' frequencies
            adgs_arr = []
            for blobed_tags in blobed_word.tags:
                if blobed_tags[1] in ['JJ', 'JJR', 'JJS']:
                    adgs_arr.append(blobed_tags[0])
            adg_frequency = {}
            for adg in adgs_arr: 
                if adg in adg_frequency:
                    adg_frequency[adg] += 1
                else:
                    adg_frequency[adg] = 1 
            tag_sentiment['adg_frequency'] = sorted(adg_frequency.items(),key=operator.itemgetter(1),reverse=True)[:3]       
        tags_metadata.append(tag_sentiment)
        
    return tags_metadata